<a href="https://colab.research.google.com/github/FernandoElizarraras7/SIMULACION-II/blob/main/LINEA_DE_ESPERA_UN_SERVIDOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ¿QUÉ ES UNA LÍNEA DE ESPERA?
Una línea de espera (o sistema de colas) es un modelo que describe situaciones donde entidades (como clientes, paquetes o tareas) llegan a un sistema para recibir un servicio, pero si el servidor está ocupado, deben esperar en una cola hasta ser atendidos.

Este concepto se utiliza para analizar y optimizar procesos en áreas como:
*   Centros de atención (personas esperando ser atendidas).
*   Redes de computadoras (paquetes esperando ser procesados).
*   Producción industrial (productos esperando en una línea de ensamblaje).

## Componentes principales de una línea de espera:
*   Clientes: Las entidades que llegan al sistema (personas, datos, trabajos).
*   Servidor: El recurso que atiende a los clientes (cajero, CPU, máquina).
*   Cola: El lugar donde los clientes esperan si el servidor está ocupado.
*   Disciplina de servicio: El orden en que se atienden los clientes (por ejemplo, FIFO: primero en llegar, primero en salir).
*   Tasa de llegada (λ): Velocidad promedio a la que llegan los clientes.
*   Tasa de servicio (μ): Velocidad promedio a la que el servidor atiende.







In [ ]:
from random import expovariate  # Para generar tiempos exponenciales (llevadas y sevicios)
from statistics import mean
from math import inf
import plotly.graph_objects as go

# === Simulación Sistema M/M/1 ===

lamda = 4.0 # Tasa de llegadas
mu = 6.0 # Tasa de servicio
Num_Pkst = 100000 # Número de paquetes a procesar (eventos de salida)
count = 0
t = 0.0
N = 0

tll = expovariate(lamda) # Tiempo de la próxima llegada
ts = inf # Tiempo de la próxima salida
t_evento_anterior = 0.0 # Tiempo del último evento (para calcular áreas y duraciones)

tll_Data = []
ts_Data = []
r_Data = []
area = []
tiempo_por_estado = {}

tiempo_grafica = [0.0]
N_grafica = [0]

while count < Num_Pkst:
    if tll < ts: # Evento de llegada
        t = tll
        tll_Data.append(t)
        # Actualizar tiempo en N
        duracion = t - t_evento_anterior
        tiempo_por_estado[N] = tiempo_por_estado.get(N, 0) + duracion
        # Actualizar área para L
        area.append(duracion * N)
        t_evento_anterior = t
        N += 1
        tll = t + expovariate(lamda) # Programar próxima llegada
        if N == 1:
            ts = t + expovariate(mu) # Si el sistema estaba vacío, programar salida
        tiempo_grafica.append(t)
        N_grafica.append(N)
    else: # Evento de salida
        t = ts
        ts_Data.append(t)
        duracion = t - t_evento_anterior
        tiempo_por_estado[N] = tiempo_por_estado.get(N, 0) + duracion
        area.append(duracion * N)
        t_evento_anterior = t
        N -= 1
        count += 1
        if N > 0:
            ts = t + expovariate(mu)
        else:
            ts = inf # Si no hay clientes, salida infinita
        tiempo_grafica.append(t)
        N_grafica.append(N)

# Cálculo de métricas
for i in range(Num_Pkst):
    d = ts_Data[i] - tll_Data[i]
    r_Data.append(d)

# Retraso promedio y número de clientes
W = round(mean(r_Data), 4)
L = round(sum(area) / t, 4)

# Cálculo de probabilidades
probabilidades = {}
tiempo_total = sum(tiempo_por_estado.values())
for estado, tiempo in tiempo_por_estado.items():
    probabilidades[estado] = tiempo / tiempo_total

# Ordenar por estado
probabilidades_ordenadas = dict(sorted(probabilidades.items()))

# Mostrar resultados
print(f"Retraso promedio (W): {W}")
print(f"Número promedio de clientes (L): {L}")
print("\nProbabilidades de N clientes en el sistema:")
for estado, prob in probabilidades_ordenadas.items():
    print(f"N = {estado}: P = {round(prob, 6)}")

# Gráfica escalonada
fig = go.Figure()
fig.add_trace(go.Scatter(x=tiempo_grafica, y=N_grafica, mode='lines', line_shape='hv',
                         name='Número de clientes', line=dict(color='blue')))
fig.update_layout(title='Evolución del número de clientes en el sistema',
                  xaxis_title='Tiempo',
                  yaxis_title='Número de clientes (N)',
                  font=dict(family='Arial', size=14))
fig.show()